## Checking PyTorch and CUDA Availability

In this code cell, we import the PyTorch library and print out information about the availability of CUDA, the CUDA version, and the PyTorch version. This is useful for verifying that PyTorch is correctly installed and that it can utilize the GPU for computations.


In [ ]:
import torch

print("CUDA Available:", torch.cuda.is_available()) 
print("CUDA Version:", torch.version.cuda)
print("PyTorch Version:", torch.__version__)


## Loading YOLOv5 Model

In this code cell, we load the YOLOv5 model from a local directory using PyTorch's hub module. After loading the model, a confirmation message is printed to indicate that the YOLOv5 model has been successfully loaded.


In [ ]:
model = torch.hub.load('D:\Object Detection\yolov5', 'yolov5n', source='local')
print("YOLOv5 model loaded successfully.")


## Checking GPU Availability and Details

This cell checks the availability of CUDA and prints the number of GPUs available. For each available GPU, it also prints its name, helping to identify the GPUs accessible for running PyTorch operations.


In [ ]:

import torch

print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


## Training the YOLOv5 Model

This cell runs the training script for the YOLOv5 model. Ensure the `data.yaml` file is correctly placed in the `yolov5` directory or provide the correct path to it. The training is configured with an image size of 640, batch size of 16, and runs for 10 epochs. The training weights are initialized with `yolov5n.pt`, and the process runs on device 0 (usually the first GPU). The results are saved in the `runs/train` directory under the name `exp_model-n_img-640`.


In [ ]:
# Correct paths if necessary
data_yaml_path = "data.yaml"  # Ensure this file is in the yolov5 directory or provide the correct path

%run train.py --img 640 --batch 16  --epochs 10 --data {data_yaml_path} --weights yolov5n.pt --device 0 --project runs/train --name exp_model-n_img-640

## Object Detection and Visualization with YOLOv5

This cell performs object detection on a set of test images using a custom-trained YOLOv5 model and visualizes the results with bounding boxes and labels.

1. **Load the YOLOv5 Model**: The model is loaded from the specified path where the custom weights are stored.
2. **Define a Function to Draw Bounding Boxes and Labels**: This function draws bounding boxes and labels on the detected objects in the image.
3. **Specify Paths**: Set the paths for the directory containing the test images and the directory where the processed images will be saved.
4. **Process Test Images**: 
    - List all image files in the test images directory.
    - For each image, apply the YOLOv5 model to detect objects.
    - Draw bounding boxes and labels on the detected objects.
    - Save the processed images to the output directory.
    - Optionally, display the processed images.

The bounding boxes and labels are drawn using OpenCV, and the processed images are saved to the specified output directory.


In [ ]:
import cv2
import torch
import numpy as np
import os

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=r'D:\Object Detection\yolov5\runs\train\exp_model-n_img-640\weights\last.pt', force_reload=True)

# Function to draw bounding boxes and labels
def draw_boxes(image, results):
    for pred in results.pred[0].detach().cpu().numpy():
        x1, y1, x2, y2, conf, cls = pred
        label = f"{model.names[int(cls)]} {conf:.2f}"
        
        # Draw bounding box
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Change thickness here (last parameter)

        # Draw label
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)  # Adjust font size (third parameter) and thickness (fourth parameter)
        cv2.rectangle(image, (int(x1), int(y1) - 20), (int(x1) + w, int(y1)), (0, 255, 0), -1)
        cv2.putText(image, label, (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)  # Adjust font size (third parameter) and thickness (fourth parameter)

# Path to test images
test_images_dir = r"D:\Object Detection\data_test"  # Use raw string notation
output_dir = r'D:\Object Detection\output_images_test_set'  # Use raw string notation

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# List all image files in the test images directory
image_files = [f for f in os.listdir(test_images_dir) if f.endswith(('.jpg', '.png'))]

# Debugging: Print the directory and files being processed
print(f"Processing images from directory: {test_images_dir}")
print(f"Found image files: {image_files}")

for image_file in image_files:
    image_path = os.path.join(test_images_dir, image_file)
    image = cv2.imread(image_path)

    # Debugging: Print the current image path being processed
    print(f"Processing image: {image_path}")

    # Apply the model to the image
    results = model(image)

    # Draw bounding boxes and labels
    draw_boxes(image, results)

    # Save the processed image
    output_image_path = os.path.join(output_dir, image_file)
    cv2.imwrite(output_image_path, image)

    # Display the image (optional)
    cv2.imshow('Processed Image', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
